In [1]:
import pandas as pd
import numpy as np

import os
import json
import time
import re

import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, SQLContext

from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import pandas_udf
from pyspark.sql.window import Window

from datetime import date, datetime, timedelta, timezone

In [2]:
appName = "PySpark TFT puuids"
master = "local[10]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.executor.memory", "40g") \
    .set("spark.driver.memory", "40g") \
    .set("spark.executor.memoryOverhead", "8g") \
    .set("spark.local.dir", "/home/mai/spark-temp") \
    .set("spark.sql.session.timeZone", "UTC") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.minExecutors", "2") \
    .set("spark.dynamicAllocation.maxExecutors", "50") \
    .set("spark.speculation", "true") 
   
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

25/03/14 15:45:45 WARN Utils: Your hostname, LAPTOP-4O0SI9BK resolves to a loopback address: 127.0.1.1; using 172.21.71.169 instead (on interface eth0)
25/03/14 15:45:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 15:45:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 15:45:47 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
ALL_FEATURES_DIR = './data/all_features/'

In [4]:
# parameters of region, tier, division
REGION = 'kr'
TIER = 'GOLD'
DIVISION = 'I'

In [5]:
df = spark.read.parquet(os.path.join(ALL_FEATURES_DIR, f'all_features_{REGION}_{TIER}_{DIVISION}.parquet'))

In [6]:
df.printSchema()

root
 |-- metadata_match_id: string (nullable = true)
 |-- puuid: string (nullable = true)
 |-- info_game_length: double (nullable = true)
 |-- gold_left: long (nullable = true)
 |-- last_round: long (nullable = true)
 |-- level: long (nullable = true)
 |-- placement: long (nullable = true)
 |-- players_eliminated: long (nullable = true)
 |-- total_damage_to_players: long (nullable = true)
 |-- win: integer (nullable = true)
 |-- max_level_by_match: long (nullable = true)
 |-- min_level_by_match: long (nullable = true)
 |-- avg_level_by_match: double (nullable = true)
 |-- diff_max_level: long (nullable = true)
 |-- diff_min_level: long (nullable = true)
 |-- max_players_eliminated_by_match: long (nullable = true)
 |-- min_players_eliminated_by_match: long (nullable = true)
 |-- avg_players_eliminated_by_match: double (nullable = true)
 |-- diff_max_players_eliminated: long (nullable = true)
 |-- diff_min_players_eliminated: long (nullable = true)
 |-- max_total_damage_to_players_by_ma